In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.utils.data as data

from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import time
import requests


In [2]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "VGG19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256,"M", 512, 512, 512, 512,"M", 512, 512, 512, 512,"M"]
}

In [3]:
class VGG_net(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG16"])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

In [4]:
device = torch.device('cuda:0')
print(device)

cuda:0


In [5]:
model=VGG_net(in_channels=3, num_classes=2).to(device)

In [6]:
PATH = 'E:\Anaconda\envs\dataset\cifar_net.pth'
model.load_state_dict(torch.load(PATH))
model.eval()

VGG_net(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3),

In [7]:
# import requests

# URL = "http://localhost:8080/report/get"
# r = requests.get(url = URL)
# data = r.json()
# print(data)

None


In [123]:
# image = mpimg.imread("E:/coll project/my_pro/backend/frontend/public/uploads/"+data["image"])
# image.shape

(168, 299, 3)

In [124]:
# train_dir = "E:/coll project/my_pro/backend/frontend/public/uploads/"+data["image"]

In [8]:
TRANSFORM_IMG =transforms.Compose([
#     transforms.Lambda(crop224),
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5,0.5,0.5],
#                          std=[0.5,0.5,0.5])
])

In [126]:
# c_img=Image.fromarray(np.uint8(image)).convert('RGB')

In [127]:
# ct_img=TRANSFORM_IMG(c_img).to(device)
# ct_img
# # plt.imshow(ct_img.permute(1,2,0))
# # plt.show()
# ct_img=ct_img.reshape(1, 3, 224, 224)
# ct_img.shape

torch.Size([1, 3, 224, 224])

In [129]:
# outputs = model(ct_img)

# _, predicted = torch.max(outputs.data, 1)

In [9]:
classes= ('Monkeypox','Normal')

In [131]:
# classes[predicted]

'Monkeypox'

In [139]:
# URL = "http://localhost:8080/report/updateName/"+data['_id']+'/'+classes[predicted]
# r = requests.get(url = URL,params = PARAMS)

In [17]:
while True:
    URL = "http://localhost:8080/report/get"
    r = requests.get(url = URL)
    data = r.json()

  
    if(data != None):
        print(data)
        image = mpimg.imread("E:/coll project/my_pro/backend/frontend/public/uploads/"+data["image"])
        c_img=Image.fromarray(np.uint8(image)).convert('RGB')
        ct_img=TRANSFORM_IMG(c_img).to(device)
        ct_img=ct_img.reshape(1, 3, 224, 224)
        outputs = model(ct_img)
        _, predicted = torch.max(outputs.data, 1)
        URL = "http://localhost:8080/report/updateName/"+data['_id']+'/'+classes[predicted]
        r = requests.get(url = URL)
    
    time.sleep(6)





KeyboardInterrupt: 